Goal: separate the ingrdients in the FoodKG table into a list of ingredients

In [1]:
# !pip install --upgrade pip -q
# !pip install psycopg2-binary -q
# !pip install SQLAlchemy -q
# !pip install spacy -q
# !python -m spacy download en_core_web_sm -q

In [2]:
import json
import os

import pandas as pd
from dotenv import load_dotenv

from util import process_food_kg_df, assemble_food_kg_df

# from sqlalchemy import create_engine

load_dotenv()  # Take environment variables from .env
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",  # local model
    api_key=api_key
)

In [3]:
# # Credentials
# host = os.getenv("POSTGRES_HOST")
# port = os.getenv("POSTGRES_PORT")
# database = os.getenv("POSTGRES_DATABASE")
# user = os.getenv("POSTGRES_USER")
# password = os.getenv("POSTGRES_PASSWORD")
# # Create the connection engine
# engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")
# # Test the connection
# try:
#     with engine.connect() as conn:
#         print("✅ Connected successfully!")
# except Exception as e:
#     print("❌ Connection failed:", e)

In [4]:
# # Test: query 5 rows
# query = "SELECT * FROM \"FoodKG\" LIMIT 5;"  # quotes protect uppercase table names
# df = pd.read_sql(query, engine)
# # display nicely
# pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.max_rows', 5)  # show up to 5 rows
# display(df)

In [5]:
df = pd.read_csv("data/raw/FoodKG_ingredients.csv")

In [6]:
df.head()

,id,title,ingredients
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva..."
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ..."
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg..."
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans..."
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ..."


In [7]:
process_food_kg_df(
    df=df,
    model="qwen/qwen3-4b-2507",
    # model="openai/gpt-oss-20b",  # 86.69% accuracy
    client=client,
    restart=True,  # reset the checkpoints
    stop_at=100
)

Wrote part_0_99.csv (0:100) — checkpoint=99
Done.


In [8]:
final_df = assemble_food_kg_df()

In [9]:
len(final_df)

100

In [10]:
# CSV can’t store native Python lists (they are objects), so use JSON dump
ingredients_table = pd.DataFrame({
    "id": final_df["id"],
    "title": final_df["title"],
    "ingredients": final_df["ingredients"],  # new, kept upon request
    "ingredients_normalized": final_df["ingredients_normalized"].apply(json.dumps),
}).sort_values("id")
# Create directory if it doesn't exist and save
os.makedirs("data/sample", exist_ok=True)
ingredients_table.to_csv("sample/FoodKG_ingredients_normalized.csv", index=False)

In [11]:
ingredients_table.tail()

,id,title,ingredients,ingredients_normalized
95,95,Baked Beans,"['3 (1 lb.) cans pork and beans', '1/2 c. bell...","""['pork and beans', 'bell pepper', 'onion', 'c..."
96,96,Sweet-N-Sour Chicken,"['2 c. diced cooked chicken', '2 Tbsp. shorten...","""['chicken', 'shortening', 'onion', 'carrot', ..."
97,97,Ambrosia,"['8 to 10 juicy oranges, peeled and diced', '1...","""['orange', 'coconut', 'pecan', 'cherry', 'sug..."
98,98,Crazy Peanut Butter Cookies,"['1 c. creamy peanut butter', '1 c. sugar', '1...","""['peanut butter', 'sugar', 'egg']"""
99,99,Brickle Bars,"['1 box yellow cake mix', '2 eggs', '1/3 c. so...","""['yellow cake mix', 'egg', 'soft margarine', ..."


In [12]:
final_df.head()

,id,title,ingredients,ingredients_normalized
0,0,No-Bake Nut Cookies,"['1 c. firmly packed brown sugar', '1/2 c. eva...","['brown sugar', 'evaporated milk', 'vanilla', ..."
1,1,Jewell Ball'S Chicken,"['1 small jar chipped beef, cut up', '4 boned ...","['chipped beef', 'chicken breast', 'cream of m..."
2,2,Creamy Corn,"['2 (16 oz.) pkg. frozen corn', '1 (8 oz.) pkg...","['corn', 'cream cheese', 'butter', 'garlic pow..."
3,3,Chicken Funny,"['1 large whole chicken', '2 (10 1/2 oz.) cans...","['chicken', 'chicken gravy', 'mushroom soup', ..."
4,4,Reeses Cups(Candy),"['1 c. peanut butter', '3/4 c. graham cracker ...","['peanut butter', 'graham cracker crumbs', 'bu..."
